In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import time
from scipy.stats import norm, multivariate_normal, uniform, invgamma
from tqdm import tqdm

from dataGen import generate_data

# Function for particle filtering with linear dynamical system
def motion_model(particles, A, Q):
    num_particles = len(particles)
    noise = np.random.normal(np.zeros(1), Q, size=num_particles)
    particles = np.dot(A, particles.T).T + noise
    return particles

def measurement_model(particles, C, R):
    num_particles = len(particles)
    noise = np.random.multivariate_normal(np.zeros(2), R, size=num_particles)
    measurements = np.dot(C.T, np.reshape(particles, (1, num_particles))).T + noise
    return measurements

def likelihoods_est(y_hat, y):
    return -0.5 * np.sum((y_hat - y)**2, axis=1)
    

def resample(particles, weights):
    num_particles = len(particles)
    indices = np.random.choice(range(num_particles), size=num_particles, replace=True, p=weights)
    resampled_particles = particles[indices]
    resampled_weights = np.ones(num_particles) / num_particles
    return resampled_particles, resampled_weights

def particle_filter(observations, A, Q, C, R, initial_particles):
    T = len(observations)
    num_particles = len(initial_particles)
    particles = initial_particles.copy()
    weights_ = np.zeros((num_particles, T))
    weights = np.ones(num_particles) / num_particles
    weights_[:,0] = weights
    x_hat = []
    
    logLikelihood = 0
   
    for t in range(1,T):
        # Motion update
        particles = motion_model(particles, A, Q)
        
        # Measurement update
        measurements_predicted = measurement_model(particles, C, R)
        likelihoods = likelihoods_est(measurements_predicted, observations[t])#-0.5 * np.sum((measurements_predicted - observations[t])**2, axis=1)
        weights_[:,t] = np.log(weights_[:,t-1]) + likelihoods
        max_weight = np.max(weights_[:,t])
        weights_[:,t] = np.exp(weights_[:,t] - max_weight)
        weights_[:,t] = weights_[:,t]/np.sum(weights_[:,t])
        #weights += likelihoods
        #max_weight = max(weights)
        #weights += max_weight
        #weights = np.exp(weights)
        #weights /= np.sum(weights)
        
        # Resampling
        #if 1 / np.sum(weights**2) < num_particles / 2:
        #particles, weights = resample(particles, weights)
        particles, weights_[:,t] = resample(particles, weights_[:,t])
        
        # Estimate the current state
        #estimated_state = np.sum(particles * weights, axis=None)
        estimated_state = np.sum(particles * weights_[:,t], axis=None)
        x_hat.append(estimated_state)
        predictiveLikelihood = max_weight + np.log(np.sum(weights_[:,t])) - np.log(num_particles)
        logLikelihood += predictiveLikelihood
        
    return x_hat, logLikelihood, weights_   

In [ ]:
def particle_marginal_metropolis_hastings(observations, 
                                          init_A, init_Q, 
                                          init_c1, 
                                          init_c2, 
                                          init_r, 
                                          initial_particles, 
                                          num_iterations, 
                                          stepsize=0.5):
    T = len(observations)
    num_particles = len(initial_particles)
    particles = initial_particles.copy()
    logLikelihoods = np.zeros(num_iterations)
    
    A = np.zeros(num_iterations)
    A_proposed = np.zeros(num_iterations)
    
    Q = np.zeros(num_iterations)
    Q_proposed = np.zeros(num_iterations)
    
    r = np.zeros(num_iterations)
    r_proposed = np.zeros(num_iterations)
    
    c1 = np.zeros(num_iterations)
    c2 = np.zeros(num_iterations)
    c1_proposed = np.zeros(num_iterations)
    c2_proposed = np.zeros(num_iterations)

    # Step 1
    A[0] = init_A
    Q[0] = init_Q
    r[0] = init_r
    R_ = np.array([[init_r, 0], [0, init_r]])
    C_ = np.array([[init_c1, init_c2]])
    estimated_state, logLikelihoods[0], _ = particle_filter(observations, A[0], Q[0], C_, R_, initial_particles)
    
    for iteration in range(1, num_iterations):
        # Sample a new parameter
        A_proposed[iteration] = max(0.0, np.random.normal(A[iteration - 1], stepsize, 1))
        Q_proposed[iteration] = max(0.0, np.random.normal(Q[iteration - 1], stepsize, 1))
        r_proposed[iteration] = max(0.0, np.random.normal(r[iteration - 1], stepsize, 1))
        c1_proposed[iteration] = max(0.0, np.random.normal(c1[iteration - 1], stepsize, 1))
        c2_proposed[iteration] = max(0.0, np.random.normal(c2[iteration - 1], stepsize, 1))
        
        R_ = np.array([[r_proposed[iteration], 0], [0, r_proposed[iteration]]])
        C_ = np.array([[c1_proposed[iteration], c2_proposed[iteration]]])

        # Apply particle filter
        estimated_state, logLikelihoods[iteration], weights = particle_filter(observations, 
                                                                              A_proposed[iteration], 
                                                                              Q_proposed[iteration], 
                                                                              C_,
                                                                              R_, 
                                                                              initial_particles)
        # Calculate the acceptance probability
        log_acceptance_prob = (logLikelihoods[iteration] + 
                               uniform.logpdf(A_proposed[iteration], 0, 3) + 
                               uniform.logpdf(Q_proposed[iteration], 0, 5) + 
                               uniform.logpdf(r_proposed[iteration], 0, 2) + 
                               uniform.logpdf(c1_proposed[iteration], 0, 2) + 
                               uniform.logpdf(c2_proposed[iteration], 0, 2)) -\
                              (logLikelihoods[iteration - 1] + 
                               uniform.logpdf(A[iteration - 1], 0, 3) + 
                               uniform.logpdf(Q[iteration - 1], 0, 5) +
                               uniform.logpdf(r[iteration - 1], 0, 2) + 
                               uniform.logpdf(c1[iteration - 1], 0, 2) + 
                               uniform.logpdf(c2[iteration - 1], 0, 2))

        acceptance_prob = np.min((1.0, np.exp(log_acceptance_prob)))

        if np.random.uniform() < acceptance_prob:
            A[iteration] = A_proposed[iteration]
            Q[iteration] = Q_proposed[iteration]
            r[iteration] = r_proposed[iteration]
            c1[iteration] = c1_proposed[iteration]
            c2[iteration] = c2_proposed[iteration]
        else:
            A[iteration] = A[iteration - 1]
            Q[iteration] = Q[iteration - 1]
            r[iteration] = r[iteration - 1]
            c1[iteration] = c1[iteration - 1]
            c2[iteration] = c2[iteration - 1]

    
    return A, Q, r, c1, c2

In [ ]:
param_state_est = True

# Define parameters
T = 100  # Number of time steps
A = .9  # Transition matrix
Q = 3.2  # Covariance matrix of the state noise
C = np.array([[1.5, 0.6]])  # Observation matrix
R = np.array([[1, 0], [0, 1]])  # Covariance matrix of the observation noise
initial_state = np.array([np.random.uniform(-1,-1)])#np.array([0])  # Initial state
initial_covariance = np.array([[1]])  # Initial state covariance
initial_particles = np.random.normal(0, 1, size=50)  # Initial particles
num_iterations = 20000  # Number of PMMH iterations

# Generate synthetic data
latent_states, observations = generate_data(T, A, Q, C, R, initial_state)

if not param_state_est:
    # Apply particle filter
    estimated_state, loglik, _ = particle_filter(observations, A, Q, C, R, initial_particles)

    plt.plot(latent_states)
    plt.plot(estimated_state)
    #plt.show()
else:
    # Apply particle marginal Metropolis-Hastings algorithm
    A_init = .1
    Q_init = .1
    r_init = .1
    c1_init = .1
    c2_init = .1
    start = time.time()
    estimated_A, estimated_Q, estimated_r, estimated_c1, estimated_c2 = particle_marginal_metropolis_hastings(
        observations, 
        A_init, 
        Q_init, 
        c1_init, 
        c2_init, 
        r_init, 
        initial_particles, 
        num_iterations
    )
    print("Elapsed time:",(time.time() - start)/60,"minutes")


In [ ]:
# Create a figure and two subplots side by side
fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(5, 1)

# Create a figure with a square shape
fig = plt.figure(figsize=(16, 16))

ax1.plot(estimated_A)
ax1.set_title('A_est')

ax2.plot(estimated_Q)
ax2.set_title('Q_est')

ax3.plot(estimated_r)
ax3.set_title('r_est')

ax4.plot(estimated_c1)
ax4.set_title('c1_est')

ax5.plot(estimated_c2)
ax5.set_title('c2_est')

# Adjust the spacing between subplots
plt.subplots_adjust(hspace=10)
#plt.plot(estimated_Q)

In [ ]:
(np.mean(estimated_A[(num_iterations//2+1):]),
 np.mean(estimated_Q[(num_iterations//2+1):]), 
 np.mean(estimated_r[(num_iterations//2+1):]),
 np.mean(estimated_c1[(num_iterations//2+1):]), 
 np.mean(estimated_c2[(num_iterations//2+1):]))